TP Request Load Test (1000 Requests) Code of  of all language

Pyhton - Load Test

In [ ]:
import requests
import time
import psutil
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count

NUM_REQUESTS = 1000
THREADS = min(100, cpu_count() * 2)  # Cap threads to avoid resource exhaustion
TARGET_URL = "https://jsonplaceholder.typicode.com/posts/1"

def send_request():
    try:
        response = requests.get(TARGET_URL, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def main():
    # Measure initial memory
    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / 1024 / 1024  # MB
    
    # Send requests concurrently
    start_time = time.time()
    start_cpu = time.process_time()
    with ThreadPoolExecutor(max_workers=THREADS) as executor:
        results = list(executor.map(lambda _: send_request(), range(NUM_REQUESTS)))
    end_cpu = time.process_time()
    end_time = time.time()
    
    # Measure final memory
    mem_after = process.memory_info().rss / 1024 / 1024  # MB
    max_memory = max(mem_before, mem_after)
    
    # Calculate metrics
    success_count = sum(1 for r in results if r)
    failure_count = NUM_REQUESTS - success_count
    wall_time = end_time - start_time
    throughput = NUM_REQUESTS / wall_time if wall_time > 0 else 0
    
    # Output metrics
    print("\n--- Load Test Report ---")
    print(f"Total requests      : {NUM_REQUESTS}")
    print(f"Successful requests : {success_count}")
    print(f"Failed requests     : {failure_count}")
    print(f"Wall-clock time    : {wall_time:.2f} seconds")
    print(f"CPU time           : {end_cpu - start_cpu:.2f} seconds")
    print(f"Max memory         : {max_memory:.2f} MB")
    print(f"Throughput         : {throughput:.2f} requests/second")

if __name__ == "__main__":
    main()

Java - Load Test

In [ ]:
package load;

import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.net.URI;
import java.time.Duration;
import java.util.concurrent.*;
import java.util.*;
import java.lang.management.ManagementFactory;
import java.lang.management.ThreadMXBean;

public class LoadTest {
    private static final int NUM_REQUESTS = 1000;
    private static final int THREADS = 100;
    private static final String TARGET_URL = "https://jsonplaceholder.typicode.com/posts/1";

    public static void main(String[] args) throws Exception {
        // Initialize HttpClient
        HttpClient client = HttpClient.newBuilder()
                .connectTimeout(Duration.ofSeconds(5))
                .followRedirects(HttpClient.Redirect.ALWAYS)
                .build();

        // Memory before
        Runtime runtime = Runtime.getRuntime();
        runtime.gc();
        long memBefore = runtime.totalMemory() - runtime.freeMemory();

        // CPU time before
        ThreadMXBean bean = ManagementFactory.getThreadMXBean();
        long startCpuTime = 0;
        long[] startThreadIds = bean.getAllThreadIds();
        for (long id : startThreadIds) {
            long time = bean.getThreadCpuTime(id);
            if (time != -1) startCpuTime += time;
        }

        // Wall-clock start
        long startTime = System.nanoTime();

        // Send requests
        ExecutorService executor = Executors.newFixedThreadPool(THREADS);
        List<Future<Boolean>> futures = new ArrayList<>();
        HttpRequest request = HttpRequest.newBuilder()
                .uri(new URI(TARGET_URL))
                .timeout(Duration.ofSeconds(5))
                .GET()
                .build();

        for (int i = 0; i < NUM_REQUESTS; i++) {
            if (i % THREADS == 0) Thread.sleep(1); // Light pacing
            futures.add(executor.submit(() -> {
                try {
                    HttpResponse<Void> response = client.send(request, HttpResponse.BodyHandlers.discarding());
                    return response.statusCode() == 200;
                } catch (Exception e) {
                    return false;
                }
            }));
        }

        int successCount = 0;
        for (Future<Boolean> fut : futures) {
            if (fut.get()) successCount++;
        }
        executor.shutdown();

        // Wall-clock end
        long endTime = System.nanoTime();

        // CPU time after
        long endCpuTime = 0;
        long[] endThreadIds = bean.getAllThreadIds();
        for (long id : endThreadIds) {
            long time = bean.getThreadCpuTime(id);
            if (time != -1) endCpuTime += time;
        }

        // Memory after
        runtime.gc();
        long memAfter = runtime.totalMemory() - runtime.freeMemory();

        // Metrics
        double wallTime = (endTime - startTime) / 1e9;
        double cpuTime = (endCpuTime - startCpuTime) / 1e9;
        double maxMemory = Math.max(memBefore, memAfter) / (1024.0 * 1024.0);
        double throughput = wallTime > 0 ? NUM_REQUESTS / wallTime : 0;

        // Report
        System.out.println("\n--- Load Test Report ---");
        System.out.println("Total requests      : " + NUM_REQUESTS);
        System.out.println("Successful requests : " + successCount);
        System.out.println("Failed requests     : " + (NUM_REQUESTS - successCount));
        System.out.println("Wall-clock time     : " + wallTime + " seconds");
        System.out.println("Total CPU time      : " + cpuTime + " seconds");
        System.out.println("Max memory usage    : " + maxMemory + " MB");
        System.out.println("Throughput          : " + throughput + " requests/second");
    }
}


Go - Load Test

In [ ]:
package main

import (
	"fmt"
	"net/http"
	"os"
	"runtime"
	"sync"
	"time"

	"github.com/shirou/gopsutil/process"
)

const (
	totalRequests = 1000
	concurrency   = 100 // Run 100 concurrent workers
	url           = "https://jsonplaceholder.typicode.com/posts/1"
)

func main() {
	runtime.GOMAXPROCS(runtime.NumCPU()) // Use all available CPU cores

	var wg sync.WaitGroup
	requests := make(chan struct{}, totalRequests)
	successCount := 0
	failureCount := 0
	var mu sync.Mutex

	// Track wall and CPU time
	start := time.Now()
	processInfo, _ := process.NewProcess(int32(os.Getpid()))
	cpuStart, _ := processInfo.Times()

	// Fill the channel with total requests
	for i := 0; i < totalRequests; i++ {
		requests <- struct{}{}
	}
	close(requests)

	// Worker pool
	for i := 0; i < concurrency; i++ {
		wg.Add(1)
		go func() {
			defer wg.Done()
			client := &http.Client{}

			for range requests {
				resp, err := client.Get(url)
				if err != nil {
					mu.Lock()
					failureCount++
					mu.Unlock()
					continue
				}

				// Do NOT read the entire body unless needed
				// Just close it immediately
				resp.Body.Close()

				mu.Lock()
				successCount++
				mu.Unlock()
			}
		}()
	}

	wg.Wait()

	elapsed := time.Since(start)
	cpuEnd, _ := processInfo.Times()
	memoryInfo, _ := processInfo.MemoryInfo()

	fmt.Println("--- Load Test Report ---")
	fmt.Println("Total requests     :", totalRequests)
	fmt.Println("Successful requests:", successCount)
	fmt.Println("Failed requests    :", failureCount)
	fmt.Printf("Wall-clock time    : %.3f seconds\n", elapsed.Seconds())
	fmt.Printf("CPU time           : %.3f seconds\n", cpuEnd.User-cpuStart.User)
	fmt.Printf("Memory used        : %.2f MB\n", float64(memoryInfo.RSS)/(1024*1024))
}


C++ - Load Test

In [ ]:
#include <iostream>
#include <vector>
#include <chrono>
#include <windows.h>
#include <winhttp.h>
#include <psapi.h>

#pragma comment(lib, "winhttp.lib")
#pragma comment(lib, "psapi.lib")

const int NUM_REQUESTS = 1000;
const int THREADS = 8; // Match typical CPU core count
const std::wstring TARGET_URL = L"https://jsonplaceholder.typicode.com/posts/1";
const DWORD STACK_SIZE = 128 * 1024; // 128 KB stack size per thread

// Global WinHTTP session and connection
HINTERNET hSession = nullptr;
HINTERNET hConnect = nullptr;
std::wstring hostName;
std::wstring urlPath;
LONG volatile success_count = 0; // Thread-safe success counter

bool initialize_winhttp() {
    hSession = WinHttpOpen(L"LoadTest/1.0", WINHTTP_ACCESS_TYPE_DEFAULT_PROXY,
                           WINHTTP_NO_PROXY_NAME, WINHTTP_NO_PROXY_BYPASS, 0);
    if (!hSession) return false;

    URL_COMPONENTS urlComp = { sizeof(URL_COMPONENTS) };
    urlComp.dwHostNameLength = -1;
    urlComp.dwUrlPathLength = -1;

    if (!WinHttpCrackUrl(TARGET_URL.c_str(), 0, 0, &urlComp)) {
        WinHttpCloseHandle(hSession);
        return false;
    }

    hostName = std::wstring(urlComp.lpszHostName, urlComp.dwHostNameLength);
    urlPath = std::wstring(urlComp.lpszUrlPath, urlComp.dwUrlPathLength);

    hConnect = WinHttpConnect(hSession, hostName.c_str(), INTERNET_DEFAULT_HTTPS_PORT, 0);
    if (!hConnect) {
        WinHttpCloseHandle(hSession);
        return false;
    }

    return true;
}

bool send_request() {
    HINTERNET hRequest = WinHttpOpenRequest(hConnect, L"GET", urlPath.c_str(),
                                            NULL, WINHTTP_NO_REFERER,
                                            WINHTTP_DEFAULT_ACCEPT_TYPES,
                                            WINHTTP_FLAG_SECURE);
    if (!hRequest) return false;

    WinHttpSetTimeouts(hRequest, 5000, 5000, 5000, 5000);

    bool success = false;

    if (WinHttpSendRequest(hRequest, WINHTTP_NO_ADDITIONAL_HEADERS, 0,
                           WINHTTP_NO_REQUEST_DATA, 0, 0, 0)) {
        if (WinHttpReceiveResponse(hRequest, NULL)) {
            DWORD statusCode = 0;
            DWORD size = sizeof(statusCode);
            WinHttpQueryHeaders(hRequest,
                                WINHTTP_QUERY_STATUS_CODE | WINHTTP_QUERY_FLAG_NUMBER,
                                NULL, &statusCode, &size, NULL);
            success = (statusCode == 200);
        }
    }

    WinHttpCloseHandle(hRequest);
    return success;
}

struct ThreadData {
    int num_requests;
};

DWORD WINAPI thread_proc(LPVOID param) {
    ThreadData* data = static_cast<ThreadData*>(param);
    for (int i = 0; i < data->num_requests; ++i) {
        if (send_request()) {
            InterlockedIncrement(&success_count);
        }
    }
    return 0;
}

long long get_memory_usage() {
    PROCESS_MEMORY_COUNTERS pmc;
    GetProcessMemoryInfo(GetCurrentProcess(), &pmc, sizeof(pmc));
    return pmc.PeakWorkingSetSize;
}

long long get_cpu_time() {
    FILETIME creationTime, exitTime, kernelTime, userTime;
    GetProcessTimes(GetCurrentProcess(), &creationTime, &exitTime, &kernelTime, &userTime);

    ULARGE_INTEGER kernel, user;
    kernel.LowPart = kernelTime.dwLowDateTime;
    kernel.HighPart = kernelTime.dwHighDateTime;
    user.LowPart = userTime.dwLowDateTime;
    user.HighPart = userTime.dwHighDateTime;

    // Return time in seconds
    return (kernel.QuadPart + user.QuadPart) / 10000000;
}

int main() {
    if (!initialize_winhttp()) {
        std::cerr << "Failed to initialize WinHTTP\n";
        return 1;
    }

    // Measure memory before
    EmptyWorkingSet(GetCurrentProcess());
    long long mem_before = get_memory_usage();

    // Start time
    auto start_time = std::chrono::high_resolution_clock::now();
    long long start_cpu = get_cpu_time();

    // Thread setup
    std::vector<HANDLE> threads(THREADS);
    std::vector<ThreadData> thread_data(THREADS);
    int requests_per_thread = NUM_REQUESTS / THREADS;
    int extra_requests = NUM_REQUESTS % THREADS;

    for (int i = 0; i < THREADS; ++i) {
        thread_data[i].num_requests = requests_per_thread + (i < extra_requests ? 1 : 0);
        threads[i] = CreateThread(NULL, STACK_SIZE, thread_proc, &thread_data[i], 0, NULL);
        if (!threads[i]) {
            std::cerr << "Failed to create thread " << i << "\n";
        }
    }

    WaitForMultipleObjects(THREADS, threads.data(), TRUE, INFINITE);
    for (int i = 0; i < THREADS; ++i) {
        CloseHandle(threads[i]);
    }

    // End time
    auto end_time = std::chrono::high_resolution_clock::now();
    long long end_cpu = get_cpu_time();
    EmptyWorkingSet(GetCurrentProcess());
    long long mem_after = get_memory_usage();

    WinHttpCloseHandle(hConnect);
    WinHttpCloseHandle(hSession);

    // Metrics
    double wall_time = std::chrono::duration<double>(end_time - start_time).count();
    double cpu_time = static_cast<double>(end_cpu - start_cpu);
    double max_memory = std::max(mem_before, mem_after) / (1024.0 * 1024.0);
    double throughput = wall_time > 0 ? NUM_REQUESTS / wall_time : 0;

    std::cout.setf(std::ios::fixed);
    std::cout.precision(6);
    std::cout << "\n--- Load Test Report ---\n";
    std::cout << "Total requests      : " << NUM_REQUESTS << "\n";
    std::cout << "Successful requests : " << success_count << "\n";
    std::cout << "Failed requests     : " << (NUM_REQUESTS - success_count) << "\n";
    std::cout << "Wall-clock time     : " << wall_time << " seconds\n";
    std::cout << "CPU time            : " << cpu_time << " seconds\n";
    std::cout << "Max memory          : " << max_memory << " MB\n";
    std::cout << "Throughput          : " << throughput << " requests/second\n";

    return 0;
}
